In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
#!pip install tensorflow==2.0.0-alpha0 
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
tf.__version__

'2.0.0-dev20190602'

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data("mnist.npz")
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [5]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [6]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [8]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [9]:
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [10]:
EPOCHS = 20

for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

Epoch 1, Loss: 0.14078624546527863, Accuracy: 95.77999877929688, Test Loss: 0.06723157316446304, Test Accuracy: 97.89999389648438
Epoch 2, Loss: 0.0918632298707962, Accuracy: 97.22000122070312, Test Loss: 0.06178327649831772, Test Accuracy: 98.04000091552734
Epoch 3, Loss: 0.06773622334003448, Accuracy: 97.9383316040039, Test Loss: 0.0604030042886734, Test Accuracy: 98.09667205810547
Epoch 4, Loss: 0.05395057424902916, Accuracy: 98.35333251953125, Test Loss: 0.06481972336769104, Test Accuracy: 98.0425033569336
Epoch 5, Loss: 0.04470845311880112, Accuracy: 98.63099670410156, Test Loss: 0.07327750325202942, Test Accuracy: 97.9780044555664
Epoch 6, Loss: 0.038550302386283875, Accuracy: 98.81805419921875, Test Loss: 0.07292838394641876, Test Accuracy: 98.05999755859375
Epoch 7, Loss: 0.03391440957784653, Accuracy: 98.9566650390625, Test Loss: 0.07330296188592911, Test Accuracy: 98.08856964111328
Epoch 8, Loss: 0.030216414481401443, Accuracy: 99.069580078125, Test Loss: 0.07421676814556122,